In [2]:
import pandas as pd
import numpy as np
import requests
import grequests
import os.path
import tensorflow as tf

In [3]:
# shamelessly taken from https://danijar.com/variable-sequence-lengths-in-tensorflow/
def getSequenceLengths(batchOfSequences):
  used = tf.sign(tf.reduce_max(tf.abs(batchOfSequences), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

In [4]:
flags = tf.flags
logging = tf.logging

In [5]:
flags.DEFINE_bool("train", True, "train mode (test mode if not)")
FLAGS = flags.FLAGS

In [6]:
tf.logging.set_verbosity(tf.logging.INFO)

In [124]:
df = pd.read_pickle('final_processed_invoker_data')
invokerItems = np.load('invokerItems.npy')

True


In [8]:
# 265 is highest ID, and we need a zero for empty
DIM = 266
batchSize = int(len(df) * 1.0 /20)
batched_ItemData = tf.placeholder(tf.float32, shape=[None, None, DIM])

batched_ItemData = tf.pad(batched_ItemData, [[0, 0], [0, 1], [0, 0]], "CONSTANT")
# shift each sequence to the left by one (stride to the right by 1)
# and then pad at the end to maintain same lengths!
batched_ItemData_strided = tf.strided_slice(batched_ItemData, [0, 1, 0], tf.shape(batched_ItemData))
targets = tf.pad(batched_ItemData_strided, [[0, 0], [0, 1], [0, 0]], "CONSTANT")
MAX_SequenceLength = batched_ItemData.shape[1]
batched_ItemDataSequenceLengths = getSequenceLengths(batched_ItemData)
lstmHiddenSize = 50
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmHiddenSize)

In [20]:
state = tf.Variable(lstmCell.zero_state(batchSize, tf.float32), trainable=False)

lstmOutput, last_states = tf.nn.dynamic_rnn(
    cell=lstmCell,
    dtype=tf.float32,
    sequence_length=batched_ItemDataSequenceLengths,
    inputs=batched_ItemData,
    initial_state = tf.nn.rnn_cell.LSTMStateTuple(state[0], state[1]))

with tf.control_dependencies([state.assign(last_states)]):
    lstmOutput = tf.identity(lstmOutput)

Tensor("strided_slice_5:0", shape=(50,), dtype=float32)


In [10]:
lstmOutput = tf.reshape(lstmOutput, [-1, lstmHiddenSize])

In [ ]:
logits = tf.matmul(lstmOutput, weight) + bias
targets = tf.reshape(targets, [-1, DIM])
mask = tf.sign(tf.reduce_max(tf.abs(targets), 1))
loss = tf.nn.softmax_cross_entropy_with_logits(labels = targets, logits = logits)
loss = mask * loss
loss = loss.reshape(loss, [batchSize, MAX_SequenceLength])
mean_loss = tf.reduce_sum(loss, reduction_indices=1) / batchSequenceLengths
cost = tf.reduce_mean(mean_loss)

In [ ]:
# default of 0.001
learning_rate = 0.001
# default of 0.99
beta1 = 0.99
# default of 0.999
beta2 = 0.999
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1, beta2 = beta2)

gradients, variables = zip(*optimizer.compute_gradients(cost))
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
train_op = optimizer.apply_gradients(zip(gradients, variables))

In [ ]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
ckpoint_dir = os.path.join(os.getcwd(), 'model-backups/model.ckpt')

In [ ]:
def train(sess):
    numEpochs = 1000
    numBatches = 1000
    for epochIter in xrange(numEpochs):
        print('Epoch: {0}'.format(epochIter))
        if (epochIter+1) % 50 == 0:
            saver.save(sess, ckpoint_dir)
        for batchItr in xrange(numBatches):
            sess.run(train_op, feed_dict = {x : df['invokerItems'].sample(n=batchSize)})

In [ ]:
trainFlag = True
with tf.Session() as sess:
    if (trainFlag):
        sess.run(init)
        train(sess)
    else:
        print 'Doing test'

In [123]:
with tf.Session() as sess:
    invokerItems = np.load('invokerItems.npy')
    tf.constant(invokerItems)